```
pip install ipywidgets

pip install ipyleaflet
jupyter nbextension enable --py widgetsnbextension

pip install bqplot
jupyter nbextension enable --py --sys-prefix bqplot
```

In [1]:
# core
import os
import math
import json
import requests
import numpy as np
import pandas as pd
import geopandas as gpd

# widgets
import ipywidgets as wg
import ipyleaflet as mwg

import matplotlib.cm
import matplotlib.colors
import matplotlib as mpl
#import matplotlib.pyplot as plt, mpld3
mpl.rcParams['figure.figsize'] = [8, 7]

# some miscellaneous settings
smap_res = 9
basemap = mwg.basemap_to_tiles(mwg.basemaps.Esri.WorldImagery)

## test

In [2]:
# read and view GeoDataFrame
gdf = gpd.GeoDataFrame.from_file("sites/Sites_lf.shp")

gdfgeog = gdf.to_crs({'init': 'epsg:4326'})
gdfgeog['centroid'] = [(p.y, p.x) for p in gdfgeog.centroid]
#gdfgeog['centroid'] = [(gdfgeog[g].bounds['maxy'], gdfgeog[g].centroid.x) for g in gdfgeog]


farray = np.linspace(0.0,1.0,len(gdfgeog))
gdfgeog['style'] = [{
    'color': mpl.colors.rgb2hex(d[0:3]),
    'fillColor': mpl.colors.rgb2hex(d[0:3]), 
    'weight': 1, 
    'fillOpacity': 0.5} for d in mpl.cm.Set3(farray)]

geodict = json.loads(gdfgeog.to_json())

bnds = gdfgeog.bounds
series = gdfgeog['geometry']

minx = min(bnds['minx'])
maxx = max(bnds['maxx'])
miny = min(bnds['miny'])
maxy = max(bnds['maxy'])
centroid = ((maxy+miny)/2, (maxx+minx)/2)

gdf.head(2)

,OBJECTID,RANGERDIST,REGION,FORESTNUMB,DISTRICTNU,DISTRICTOR,FORESTNAME,DISTRICTNA,GIS_ACRES,SHAPE_Leng,...,STD_27,STD_28,STD_29,STD_30,STD_31,STD_32,STD_33,STD_34,STD_35,geometry
0,61,99030501010343,03,05,01,030501,Coronado National Forest,Douglas Ranger District,434025.200,3.963602,...,843.881307,681.529060,446.146563,453.982071,557.405375,339.841942,599.562885,470.501168,343.082608,(POLYGON ((-1240840.015928872 1084453.22714483...
1,62,99030503010343,03,05,03,030503,Coronado National Forest,Sierra Vista Ranger District,321534.997,2.854066,...,482.202771,470.039302,502.615547,430.908201,701.371540,272.786808,445.029396,525.713796,242.986111,(POLYGON ((-1377218.699379425 1058461.84460117...


In [3]:
# initialize widgets
m = mwg.Map(layers=(basemap,), center=(33, -109), zoom=6, scroll_wheel_zoom=True) 
layer = mwg.GeoJSON(data=geodict, hover_style={
    'color': "white",
    'weight': 2, 
    'fillOpacity': 0.5})

def hover_handler(event=None, id=None, properties=None):
    if len(m.layers)==3:
        m.remove_layer(m.layers[-1])

    nyears = len([key for key in properties.keys() if "MEAN" in key])
    data = {'year': [], 'mean': [], 'std': []}
    for i in range(1,nyears):
        data['year'].append(1980+i)
        data['mean'].append(properties["MEAN_"+str(i)])
        data['std'].append(properties["STD_"+str(i)])
    df = pd.DataFrame(data)
        
    popup = mwg.Popup(location=properties['centroid'], 
                      child=wg.HTML("{put some info here}"), 
                      auto_close=True,
                      class_name="custom")
    m.add_layer(popup)
    
layer.on_hover(hover_handler)
m.add_layer(layer)
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …